In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

In [ ]:
# load data and keeping the needed data columns 
data = pd.read_csv('/content/Sentiment.csv')
data = data[['text', 'sentiment']]
data.head(5) # looking at the data

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower()) # lowercase text
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows(): # iterate through the data and replace the rows with rt with ' ' 
  row[0] = row[0].replace('rt', ' ')

In [ ]:
max_features = 2000 # creating max feauture variable 
tokenizer = Tokenizer(num_words=max_features, split=' ') #passing in the number of max features and splitting the data

 # getting the values fitting the tokenizer 
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) 
X = pad_sequences(X)

In [ ]:
embed_dim = 128 # embedding dimensions
lstm_out = 196 # lst output
# model created
def createmodel():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.save('model.h5') # saving the model 

    return model

model_sum = createmodel()
model_sum.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# label encoding the senteiment columns 
labelencoder = LabelEncoder() 
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) # traning the data with test size of 0.33

# compiling the model 
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
# looking at the score
print(score)
print(acc)
print(model.metrics_names)

291/291 - 46s - loss: 0.8268 - accuracy: 0.6451
144/144 - 3s - loss: 0.7806 - accuracy: 0.6562
0.7806485295295715
0.6561817526817322
['loss', 'accuracy']


In [ ]:
#loading model model.h5 and predictng the model using the sentence provided in the ICP
from keras.models import load_model
loaded_model = load_model('/content/model.h5')
my_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing."

new_tokenizer = Tokenizer(num_words=max_features, split=' ') #passing in the number of max features and splitting the data

 # getting the values fitting the tokenizer 
new_tokenizer.fit_on_texts(my_text) 
X = new_tokenizer.texts_to_sequences(my_text) 
X = pad_sequences(X)
loaded_model.predict([X])

array([[0.33283338, 0.3354214 , 0.3317452 ],
       [0.3357007 , 0.33065656, 0.33364278],
       [0.33096087, 0.33656436, 0.3324748 ],
       [0.33426136, 0.33163586, 0.33410278],
       [0.3340708 , 0.33195505, 0.3339742 ],
       [0.3357007 , 0.33065656, 0.33364278],
       [0.33426136, 0.33163586, 0.33410278],
       [0.33420375, 0.33065507, 0.33514115],
       [0.3357007 , 0.33065656, 0.33364278],
       [0.33417854, 0.33335453, 0.3324669 ],
       [0.33426136, 0.33163586, 0.33410278],
       [0.33426136, 0.33163586, 0.33410278],
       [0.3334536 , 0.33411723, 0.33242917],
       [0.3357007 , 0.33065656, 0.33364278],
       [0.3340708 , 0.33195505, 0.3339742 ],
       [0.33282185, 0.334315  , 0.33286315],
       [0.33263457, 0.33159095, 0.33577448],
       [0.33373022, 0.3333492 , 0.33292055],
       [0.33417854, 0.33335453, 0.3324669 ],
       [0.33441266, 0.33116743, 0.3344199 ],
       [0.3357007 , 0.33065656, 0.33364278],
       [0.33283338, 0.3354214 , 0.3317452 ],
       [0.